# Reading retinal data with phy

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import h5py
import phy
from phy.io.h5 import open_h5
%matplotlib inline

In [5]:
%ls test-circus/

checkerboard.amplitudes-merged.mat*  checkerboard.spiketimes-merged.mat*
checkerboard.basis.npz*              checkerboard.templates-merged.mat*
checkerboard.clusters-merged.mat*    mea_252.mapping.mat*


## Reading the data

### Electrodes

In [10]:
fn_elec = 'test-circus/mea_252.mapping.mat'
f_elec = open_h5(fn_elec)
f_elec.describe()

In [26]:
mapping = f_elec.read('/Mapping')[...]
positions = f_elec.read('/Positions')[...].T.astype(np.float32)

In [36]:
f_elec.close()

### Templates

In [37]:
fn_temp = 'test-circus/checkerboard.templates-merged.mat'
f_temp = open_h5(fn_temp)
f_temp.describe()

/templates                                        (1116, 61, 252)     float32 


In [38]:
templates = f_temp.read('/templates')[...]

In [40]:
templates.shape  # this is equivalent to our 'waveforms' array: (n_templates, n_samples, n_channels)

(1116, 61, 252)

In [43]:
f_temp.close()

In [47]:
n_templates, n_samples, n_elec = templates.shape

### Spiketimes

In [103]:
fn_s = 'test-circus/checkerboard.spiketimes-merged.mat'
f_s = open_h5(fn_s)
#f_s.describe()

In [104]:
spiketimes = {i: f_s.read('temp_{}'.format(i))[:].ravel() for i in range(n_templates)
              if 'temp_{}'.format(i) in f_s.children()}

In [105]:
len(spiketimes)

588

In [106]:
f_s.close()

In [107]:
spiketimes[0]

array([   1857,    5419,   13774, ..., 4325002, 4325069, 4325420], dtype=int32)

### Clusters

In [ ]:
fn_c = 'test-circus/checkerboard.clusters-merged.mat'
f_c = open_h5(fn_c)
#f_c.describe()

In [72]:
children = f_c.children()

shape = (n_spikes,) for each electrode

In [87]:
clusters = {i: f_c.read('/clusters_{}'.format(i))[:].ravel() for i in range(n_elec)
            if 'clusters_{}'.format(i) in children}

shape = (n_spikes, n_features) for each electrode

In [88]:
features = {i: f_c.read('/data_{}'.format(i))[...].T for i in range(n_elec)
            if 'data_{}'.format(i) in children}

In [92]:
f_c.close()

In [91]:
len(clusters), len(features)

(252, 252)

In [89]:
clusters[0].shape

(625,)

In [90]:
features[0].shape

(625, 140)

### Amplitudes

In [108]:
fn_a = 'test-circus/checkerboard.amplitudes-merged.mat'
f_a = open_h5(fn_a)
#f_a.describe()

In [109]:
children = f_a.children()

In [110]:
amplitudes = {i: f_a.read('temp_{}'.format(i))[...][0] for i in range(n_templates)
              if 'temp_{}'.format(i) in children}

In [111]:
amplitudes[0]

array([ 0.97597289,  0.73035991,  0.59705037, ...,  1.20012832,
        0.77556044,  0.74537909], dtype=float32)

In [113]:
len(amplitudes)

588

In [114]:
f_a.close()

List of variables:

* n_elec
* n_samples
* n_templates
* amplitudes
* clusters
* features
* mapping
* positions
* spiketimes
* templates

## Views

**Views**:

- 1 panel qui affiche les templates en fonction du mapping (défiler)
- Pour un template selectioné, un panel qui montre amplitudes vs spiketimes
- Pour une electrode donnée, un panel qui montre le nuage de points et les clusters qui ont été effectués
- si on selectionne un template, il provient d'une électrode, et on aimerait bien voir le nuage de points correspondants qui l'a généré
- les waveforms des spikes d'un tempate
- CCG